In [ ]:
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AIMessage, HumanMessage,BaseMessage
from langgraph.graph.message import add_messages

from langgraph.prebuilt import ToolNode, tools_condition
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
import requests
import random

In [ ]:
load_dotenv()

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
llm.invoke([HumanMessage(content="Hi")])

In [ ]:
#tool creation
search_tool = DuckDuckGoSearchRun(region="us-en")

@tool
def calculator(num1:float,num2:float, operation:float)->dict:
    """Perform a basic arithmetic opeation on two numbers
        supported opreations: add, subtract, multiplication, division
    """
    try:
        if operation == "add":
            result = num1 + num2
        elif operation == "sub":
            result = num1 - num2
        elif operation == "mul":
            result = num1 * num2
        elif operation == "div":
            if num2 == 0:
                return {"error": "Division by zero is not allowed"}
            result = num1 / num2
        else:
            return {"error": f"Unsupported operation '{operation}'"}
        return {"num1": num1, "num2": num2, "operation": operation, "result": result}
    except Exception as e:
        return {"error":f"{e}"}
    

In [ ]:
#make tool list
tools = [calculator,search_tool]


In [ ]:
# Make the LLM tool-aware
llm_with_tools = llm.bind_tools(tools)

In [ ]:
#state class creation
class Mystate(TypedDict):
    messages:Annotated[list[BaseMessage],add_messages]




In [ ]:
#chat node
def chat_node(state: Mystate)-> dict:
    """LLM can answer or make a tolls call"""
    message = state["messages"]
    response = llm_with_tools.invoke(message)
    return {"message":[response]}

tool_node = ToolNode(tools)

In [ ]:
#cread graph
graph_builder = StateGraph(Mystate)

graph_builder.add_node("chat_node",chat_node)
graph_builder.add_node("tools",tool_node)


In [ ]:
graph_builder.add_edge(START,"chat_node")
# If the LLM asked for a tool, go to ToolNode; else finish
graph_builder.add_conditional_edges("chat_node", tools_condition)
graph_builder.add_edge("tools", "chat_node")    


In [ ]:
chatbot = graph_builder.compile()

In [ ]:
chatbot

In [ ]:
result= chatbot.invoke({"messages":[HumanMessage(content='Hello!')]})


In [ ]:
result['messages'][-1].content

In [66]:
result = chatbot.invoke({'messages':[HumanMessage(content='multiply 3*8')]})
result

{'messages': [HumanMessage(content='multiply 3*8', additional_kwargs={}, response_metadata={}, id='b44b0f5a-71f3-4bb5-ae56-18a4571b1866')]}

In [67]:
# Regular chat
out = chatbot.invoke({"messages": [HumanMessage(content="Hello!")]})

print(out["messages"][-1].content)

ChatGoogleGenerativeAIError: Error calling model 'gemini-2.5-flash' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 35.041965767s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeT